In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
houses = ['A', 'B', 'C']
xtype_dict = {'r':'raw', 'c':'change'}

In [3]:
def read_data(house, xtype):
    assert house in houses
    assert xtype in xtype_dict
    
    act_df = pd.read_csv("house{}_act.csv".format(house))
    sensor_df = pd.read_csv("house{}_sensor.csv".format(house))
    X = np.load("X_{}_house{}.npy".format(xtype_dict[xtype], house))
    Y = np.load("Y_house{}.npy".format(house))
    return act_df, sensor_df, X, Y

In [66]:
#read in data
act_df,sensor_df,X,Y = read_data('A', 'r')

In [67]:
#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

In [68]:
print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

# sensors:  14
# states/acts:  17
# timeframes:  40006


## Naive Bayes

In [69]:
#each row represents sensors for a specific action i
#each column is sensor j
def calc_miu(X, Y, all_sensor=all_sensor, all_act=all_act):
    assert len(X)==len(Y)
    
    num_act = len(all_act)
    num_sensor = len(all_sensor)
    miu = np.zeros([num_act, num_sensor])
    for act in all_act:
        sensor = X[Y==act]
        for row in sensor:
            miu[a2i[act]] += row
    #normalize
    for i,row in enumerate(miu):
        total = np.sum(row)
        if total == 0:
            miu[i] += 1
            total = np.sum(miu[i])
        miu[i] = miu[i]/total
        
    return miu

In [70]:
def max_like(miu, X):
    num_time = len(X)
    num_act = len(miu)
    
    prob = np.zeros((num_time, num_act))
    for i in range(num_time):
        sensor = X[i]
        for j in range(num_act):
            act = miu[j]
            logp = 0
            for k,s in enumerate(sensor):
                if s == 1:
                    logp += np.log(act[k])
                else:
                    logp += np.log(1-act[k])
            prob[i,j] = logp
    return prob

In [71]:
def mle(matrix):
    est = np.zeros(len(matrix))
    for i,row in enumerate(matrix):
        est[i] = np.argmax(row)
    return est

In [72]:
miu = calc_miu(X,Y)
likes = max_like(miu,X)
result = mle(likes) #note: result is index, not label yet

In [73]:
#convert index to label
res_label = np.array([i2a[e] for e in result])
for act in all_act:
    print "act label: {}, Y count:{}, estimate count:{}".format(act, sum(Y==act), sum(res_label==act))

act label: 1.0, Y count:19723, estimate count:2610
act label: 3.0, Y count:0, estimate count:9961
act label: 4.0, Y count:220, estimate count:595
act label: 5.0, Y count:240, estimate count:8153
act label: 6.0, Y count:31, estimate count:1607
act label: 10.0, Y count:11670, estimate count:6450
act label: 13.0, Y count:71, estimate count:25
act label: 15.0, Y count:292, estimate count:3
act label: 16.0, Y count:33, estimate count:154
act label: 17.0, Y count:21, estimate count:35
act label: 18.0, Y count:38, estimate count:41
act label: 19.0, Y count:25, estimate count:40
act label: 20.0, Y count:2, estimate count:499
act label: 22.0, Y count:10, estimate count:922
act label: 23.0, Y count:11, estimate count:7111
act label: 25.0, Y count:454, estimate count:936
act label: 0, Y count:7165, estimate count:864


# Evaluation

$$\text{Precision} = \frac{1}{N}\sum^N_i \frac{TP_i}{TI_i} $$
$$\text{Recall} = \frac{1}{N}\sum^N_i\frac{TP_i}{TT_i} $$
$$\text{F-Measure} = \frac{2 \cdot precision \cdot recal}{precision+recall} $$
$$\text{Accuracy} = \frac{\sum^N_i TP_i}{Total}$$

In [74]:
def precision(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TI = np.sum(pred_label==y)
        res += (float(TP)/TI)
    return float(res)/N

def recall(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TT = np.sum(Y==y)
        res += float(TP)/TT
    return float(res)/N

def f_score(pred_label, Y):
    p = precision(pred_label, Y)
    r = recall(pred_label, Y)
    return 2*p*r/(p+r)

def accuracy(pred_label, Y):
    res = 0
    all_label = list(set(Y))
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        res += TP
    return float(res)/len(Y)

In [75]:
precision(res_label, Y)

0.3150239010523815

In [76]:
recall(res_label, Y)

0.43400960627952745

In [77]:
f_score(res_label, Y)

0.3650661764155246

In [78]:
accuracy(res_label, Y)

0.15237714342848574